In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004, 'o3': 'atmpro'}
band = [7]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.55, 0.9)]
o co2 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.039602                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.031055
0.000750      2.0 -0.043982                  2.0 -0.034514
0.001052      3.0 -0.052429                  3.0 -0.041251
0.001476      4.0 -0.060531                  4.0 -0.047742
0.002070      5.0 -0.068301                  5.0 -0.054005
0.002904      6.0 -0.075751                  6.0 -0.060054
0.004074      7.0 -0.082859                  7.0 -0.065859
0.005714      8.0 -0.089619                  8.0 -0.071427
0.008015      9.0 -0.096038                  9.0 -0.076737
0.011243     10.0 -0.102139                 10.0 -0.081811
0.015771     11.0 -0.108066                 11.0 -0.086630
0.022122     12.0 -0.113837                 12.0 -0.091226
0.031031     13.0 -0.119616                 13.0 -0.095609
0.043528     14.0 -0.125348                 14.0 -0.099664
0.061057     15.0 -0.127273                 15.0 -0.100941
0.085645     16.0 -0.117989                 16.0 -0.094571
0.120136     17.0 -0.069534                 17.0 -0.057462
0.168516     18.0  0.033677                 18.0  0.026895
0.236378     19.0  0.202039                 19.0  0.171326
0.331549     20.0  0.447926                 20.0  0.391371
0.465100     21.0  0.784813                 21.0  0.703535
0.652400     22.0  1.232713                 22.0  1.124551
0.915100     23.0  1.754147                 23.0  1.625225
1.283650     24.0  2.016286                 24.0  1.929184
1.800600     25.0  1.808280                 25.0  1.795452
2.525700     26.0  1.449857                 26.0  1.444915
3.542800     27.0  1.072546                 27.0  1.063405
4.969550     28.0  0.753432                 28.0  0.761339
6.970850     29.0  0.530250                 29.0  0.561333
9.778100     30.0  0.359931                 30.0  0.404790
13.715850    31.0  0.187314                 31.0  0.228250
19.239350    32.0  0.018676                 32.0  0.042806
26.987250    33.0 -0.099576                 33.0 -0.092240
37.855300    34.0 -0.182464                 34.0 -0.190960
53.100050    35.0 -0.252031                 35.0 -0.278060
73.887500    36.0 -0.274543                 36.0 -0.314613
97.662500    37.0 -0.243610                 37.0 -0.283626
121.437500   38.0 -0.211238                 38.0 -0.244181
145.212500   39.0 -0.196616                 39.0 -0.221422
168.987500   40.0 -0.176079                 40.0 -0.191990
192.762500   41.0 -0.141177                 41.0 -0.150303
216.537500   42.0 -0.112040                 42.0 -0.117720
240.312500   43.0 -0.092085                 43.0 -0.095667
264.087500   44.0 -0.073675                 44.0 -0.075719
287.862500   45.0 -0.059101                 45.0 -0.060194
311.637500   46.0 -0.049318                 46.0 -0.049838
335.412500   47.0 -0.040926                 47.0 -0.041027
359.187500   48.0 -0.033095                 48.0 -0.032939
382.962500   49.0 -0.026275                 49.0 -0.025988
406.737500   50.0 -0.020322                 50.0 -0.019974
430.512500   51.0 -0.014774                 51.0 -0.014412
454.287500   52.0 -0.009622                 52.0 -0.009287
478.062500   53.0 -0.004843                 53.0 -0.004553
501.837500   54.0 -0.000419                 54.0 -0.000176
525.612500   55.0  0.003806                 55.0  0.003979
549.387500   56.0  0.007950                 56.0  0.008051
573.162500   57.0  0.012095                 57.0  0.012125
596.937500   58.0  0.016554                 58.0  0.016496
620.712500   59.0  0.021189                 59.0  0.021058
644.487500   60.0  0.026000                 60.0  0.025773
668.262500   61.0  0.031020                 61.0  0.030666
692.037500   62.0  0.036228                 62.0  0.035770
715.812500   63.0  0.041685                 63.0  0.041174
739.587500   64.0  0.047431                 64.0  0.046971
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -22.506210  0.000000 -22.506210        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -22.123351  0.000002   
6.244000e-04 2     -22.506215  0.000001 -22.506213 -22.123354  0.000003   
8.759000e-04 3     -22.506217  0.000002 -22.506215 -22.123356  0.000003   
1.228600e-03 4     -22.506220  0.000003 -22.506217 -22.123358  0.000004   
1.723400e-03 5     -22.506225  0.000005 -22.506220 -22.123363  0.000006   
2.417400e-03 6     -22.506234  0.000008 -22.506226 -22.123369  0.000008   
3.390900e-03 7     -22.506248  0.000014 -22.506235 -22.123380  0.000011   
4.756500e-03 8     -22.506270  0.000022 -22.506248 -22.123396  0.000017   
6.672000e-03 9     -22.506303  0.000035 -22.506268 -22.123422  0.000026   
9.358900e-03 10    -22.506354  0.000055 -22.506299 -22.123460  0.000041   
1.312780e-02 11    -22.506431  0.000086 -22.506345 -22.123519  0.000063   
1.841450e-02 12    -22.506547  0.000135 -22.506412 -22.123608  0.000098   
2.583020e-02 13    -22.506721  0.000209 -22.506512 -22.123741  0.000151   
3.623230e-02 14    -22.506982  0.000322 -22.506660 -22.123940  0.000232   
5.082340e-02 15    -22.507369  0.000493 -22.506876 -22.124236  0.000355   
7.129060e-02 16    -22.507953  0.000768 -22.507185 -22.124681  0.000555   
1.000000e-01 17    -22.508902  0.001316 -22.507586 -22.125406  0.000959   
1.402710e-01 18    -22.510436  0.002518 -22.507918 -22.126582  0.001861   
1.967600e-01 19    -22.512709  0.005017 -22.507692 -22.128321  0.003779   
2.759970e-01 20    -22.515773  0.009977 -22.505796 -22.130627  0.007694   
3.871000e-01 21    -22.519338  0.019437 -22.499901 -22.133186  0.015404   
5.431000e-01 22    -22.522250  0.036852 -22.485398 -22.134894  0.030115   
7.617000e-01 23    -22.521227  0.067750 -22.453477 -22.132840  0.057183   
1.068500e+00 24    -22.509968  0.120241 -22.389727 -22.121631  0.105046   
1.498800e+00 25    -22.484641  0.197688 -22.286953 -22.097759  0.179518   
2.102400e+00 26    -22.448713  0.291053 -22.157659 -22.063690  0.273838   
2.949000e+00 27    -22.404814  0.392555 -22.012259 -22.021526  0.376595   
4.136600e+00 28    -22.358259  0.496885 -21.861374 -21.976248  0.480931   
5.802500e+00 29    -22.317253  0.604560 -21.712693 -21.935911  0.590851   
8.139200e+00 30    -22.289358  0.723437 -21.565921 -21.907788  0.718120   
1.141700e+01 31    -22.285775  0.859608 -21.426167 -21.902833  0.870353   
1.601470e+01 32    -22.329393  1.005243 -21.324150 -21.944055  1.035899   
2.246400e+01 33    -22.453901  1.144019 -21.309882 -22.066160  1.190710   
3.151050e+01 34    -22.694494  1.277904 -21.416590 -22.308408  1.334101   
4.420010e+01 35    -23.105433  1.414568 -21.690864 -22.734885  1.473503   
6.200000e+01 36    -23.757392  1.535114 -22.222278 -23.441687  1.593948   
8.577500e+01 37    -24.614535  1.619056 -22.995479 -24.412920  1.679040   
1.095500e+02 38    -25.352119  1.670558 -23.681561 -25.264783  1.732040   
1.333250e+02 39    -25.988122  1.711647 -24.276475 -25.994615  1.774108   
1.571000e+02 40    -26.576264  1.746056 -24.830208 -26.653779  1.809613   
1.808750e+02 41    -27.104648  1.778546 -25.326102 -27.227600  1.842671   
2.046500e+02 42    -27.541578  1.817877 -25.723701 -27.689009  1.880737   
2.284250e+02 43    -27.907987  1.868745 -26.039243 -28.068337  1.928495   
2.522000e+02 44    -28.227869  1.929285 -26.298584 -28.394241  1.984941   
2.759750e+02 45    -28.500772  1.994697 -26.506075 -28.668441  2.045871   
2.997500e+02 46    -28.736041  2.063518 -26.672523 -28.902075  2.109961   
3.235250e+02 47    -28.949084  2.137666 -26.811418 -29.111541  2.179053   
3.473000e+02 48    -29.143390  2.216711 -26.926679 -29.300751  2.252706   
3.710750e+02 49    -29.319347  2.299463 -27.019885 -29.470650  2.329829   
3.948500e+02 50    -29.480009  2.386126 -27.093883 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')